# DRM Data Prep
The purpose of this Notebook is to take data output from CALPADs on student's previous CELDT scores and manipulate it in such a way that it can be used to update the previous CELDT scores in the offline version of the CELDT Data Review Module (DRM). The DRM is a tool that allows LEAs to correct their CELDT information prior to its public release.

There are two parts to this notebook:

#### Part 1: Loading and Manipulating CALPADS records of students previous CELDT scores

Specifically, this part of the notebook does the following:
1. concatenates all of the output files with long CELDT scale score data from CALPADS from each of the schools
2. filters for the most recent CELDT result and the right columns
3. transforms the data from long to wide with one row per SSID

##### Inputs:
* Previous CELDT results files from CALPADS, one per school

##### Outputs:
output_df - This was the original stopping point of this notebook - you could use this if you wanted to vlookup the results into the excel versions of the DRM files. Upon further exploration, however, given the quirks of how Excel deals with different date types it was a lot of work to get it to save, for example, the TestTakenGrade as a two digit number (e.g. 08 for grade 8). Thus the output of the seconnd part of this file (described below) is probably more useful.
* single CSV file with all school's most CELDT results, one row per student by SSID with the following columns:
  * 'SSID'
  * 'Listening'
  * 'Overall'
  * 'Reading'
  * 'Speaking'
  * 'Writing'
  * 'SSIDAdminDate'
  * 'TestTakenGrade'
  
#### Part 2: Updating the CELDT DRM Offline Files -- Loading DRM Offline Files, Checking Against Illuminate, Adding Previous CELDT Scores, Outputting to Fixed-Width Text

Specifically, this part of the notebook does the following:
1. loads and concatenates all of the DRM offline update files for each our schools
2. provides checks for SSID integrity
3. merges in the previous CELDT result information from part 1
4. outputs this data to a fixed-width text file

##### Inputs:
* CELDT DRM Offline Files - Excel Format - From each school's CELDT Portal

##### Outputs:
final_drm_df / fw_output_list - This data frame, when saved as a text file, can be uploaded directly to the CELDT DRM Offline updater tool that is accessible through the CELDT District Portal. **Important Note:** The idea here is to create ONE file for ALL the schools and upload it to EACH of the different CELDT District Portals. The reason being is that the CELDT DRM tool matches and updates students based on their EDS unique identifiers, so it will only update the ones that it finds in our file that match its current student set.
* single fixed-width text file with 58 columns defined by the [2016-17 DRM Data File Layout]('https://www.celdt.org/documents/2016-17/CELDT_DRM_Data_File_Layout_2016-17.pdf')

In [205]:
import pandas as pd
import os
from glob import glob

In [248]:
# read in all files and concatenate them. reset the index so we can do joins easily later
frames = []

# converters used to control dtypes - we want to use all important fields as text for convenience reasons
converters = {
    'SSID': str, 
    'TestTakenGrade': str,
    'ScaleScoreAll': str}

for file in glob('./data/calpads_celdt_scores/*.xlsx'):
    dirname, filename = os.path.split(file)
    temp_df = pd.DataFrame(pd.read_excel(file, converters=converters))
    #temp_df['Application Year'] = '20' + filename[0:2] + '-' + filename[2:4]
    frames.append(temp_df)
    
calpads_celdt_df = pd.concat(frames)
calpads_celdt_df.reset_index(inplace=True)

# check inferred dtypes
calpads_celdt_df.columns.to_series().groupby(calpads_celdt_df.dtypes).groups

{dtype('<M8[ns]'): ['BirthDate',
  'SchoolStartDate',
  'ELASStartDate',
  'DueDate',
  'SSIDAdminDate'],
 dtype('int64'): ['index',
  'SchoolCode',
  'LocalID',
  'EnrlmtStatCode',
  'Grade_ODS',
  'TestTakenPurpose'],
 dtype('O'): ['SchoolName',
  'SSID',
  'StudentName',
  'LanguageName',
  'ELASCode',
  'TestPurpose_1',
  'TestStatus',
  'TestTakenGrade',
  'AcdmcYearCode',
  'LEA',
  'TestNameSpecific',
  'ScaleScoreAll',
  'ScrPerfLvlAll',
  'CriterionMet']}

In [250]:
# number of different students in the concatenated files. Need to make sure this stays constant
len(calpads_celdt_df['SSID'].unique())

287

In [251]:
# determine which CELDT testing year was most recent for each student
calpads_celdt_df['Most Recent CELDT Year'] = calpads_celdt_df.groupby(['SSID'])['AcdmcYearCode'].transform(max)

In [252]:
# only keep the CELDT scores from the most recent administration that the student has
most_recent_celdt_df = calpads_celdt_df[calpads_celdt_df['AcdmcYearCode'] == calpads_celdt_df['Most Recent CELDT Year']]

In [253]:
# get rid of the comprehension category b/c this is not uploaded in the DRM file
most_recent_celdt_df = most_recent_celdt_df[most_recent_celdt_df['TestNameSpecific'] != 'Comprehension']

In [254]:
# take a look at the df
most_recent_celdt_df

,index,SchoolCode,SchoolName,SSID,StudentName,LocalID,EnrlmtStatCode,Grade_ODS,BirthDate,SchoolStartDate,...,TestTakenGrade,TestTakenPurpose,SSIDAdminDate,AcdmcYearCode,LEA,TestNameSpecific,ScaleScoreAll,ScrPerfLvlAll,CriterionMet,Most Recent CELDT Year
1,1,128090,Summit Public School: Denali,3158782547,"Aguilar, Misael Antonio",50002,10,9,2002-11-14,2016-08-16,...,7,2,2014-09-29,2014-2015,Summit Public School: Denali,Listening,715,5-Advanced,Y,2014-2015
2,2,128090,Summit Public School: Denali,3158782547,"Aguilar, Misael Antonio",50002,10,9,2002-11-14,2016-08-16,...,7,2,2014-09-29,2014-2015,Summit Public School: Denali,Overall,603,4-Early Advanced,Y,2014-2015
3,3,128090,Summit Public School: Denali,3158782547,"Aguilar, Misael Antonio",50002,10,9,2002-11-14,2016-08-16,...,7,2,2014-09-29,2014-2015,Summit Public School: Denali,Reading,604,4-Early Advanced,Y,2014-2015
4,4,128090,Summit Public School: Denali,3158782547,"Aguilar, Misael Antonio",50002,10,9,2002-11-14,2016-08-16,...,7,2,2014-09-29,2014-2015,Summit Public School: Denali,Speaking,545,4-Early Advanced,Y,2014-2015
5,5,128090,Summit Public School: Denali,3158782547,"Aguilar, Misael Antonio",50002,10,9,2002-11-14,2016-08-16,...,7,2,2014-09-29,2014-2015,Summit Public School: Denali,Writing,551,3-Intermediate,Y,2014-2015
13,13,128090,Summit Public School: Denali,3194759507,"Agustin, Carmela Punzalan",50779,10,6,2004-10-24,2016-08-16,...,3,2,2012-08-02,2012-2013,Santa Clara Unified,Listening,535,4-Early Advanced,N,2012-2013
14,14,128090,Summit Public School: Denali,3194759507,"Agustin, Carmela Punzalan",50779,10,6,2004-10-24,2016-08-16,...,3,2,2012-08-02,2012-2013,Santa Clara Unified,Overall,497,3-Intermediate,N,2012-2013
15,15,128090,Summit Public School: Denali,3194759507,"Agustin, Carmela Punzalan",50779,10,6,2004-10-24,2016-08-16,...,3,2,2012-08-02,2012-2013,Santa Clara Unified,Reading,520,3-Intermediate,N,2012-2013
16,16,128090,Summit Public School: Denali,3194759507,"Agustin, Carmela Punzalan",50779,10,6,2004-10-24,2016-08-16,...,3,2,2012-08-02,2012-2013,Santa Clara Unified,Speaking,449,3-Intermediate,N,2012-2013
17,17,128090,Summit Public School: Denali,3194759507,"Agustin, Carmela Punzalan",50779,10,6,2004-10-24,2016-08-16,...,3,2,2012-08-02,2012-2013,Santa Clara Unified,Writing,487,3-Intermediate,N,2012-2013


In [255]:
# create an output_df that melts the long data from the CALPADS file into wide data. Keep the other values we need
output_df = pd.merge(most_recent_celdt_df.pivot(index='SSID', columns='TestNameSpecific', values='ScaleScoreAll').reset_index(),
                  most_recent_celdt_df[['SSID', 'SSIDAdminDate','TestTakenGrade']], how='left', on='SSID')

# drop any duplicates from the above merge
output_df.drop_duplicates(inplace=True)

# re-order columns to be in line with DRM files
output_df = output_df[['SSID', 'SSIDAdminDate', 'Listening', 'Speaking', 'Reading', 'Writing', 'Overall', 'TestTakenGrade']]

# check that we still have the proper number of students
len(output_df['SSID'].unique())

287

In [256]:
output_df.head()

TestNameSpecific,SSID,SSIDAdminDate,Listening,Speaking,Reading,Writing,Overall,TestTakenGrade
0,1002843922,2015-10-27,599,580,578,538,573,7
5,1075007913,2015-10-15,700,618,639,701,664,10
10,1084987293,2013-08-22,573,527,398,562,515,9
15,1096623505,2015-08-31,555,487,503,515,515,6
20,1109854923,2015-10-01,551,639,565,352,526,11


In [258]:
# final visual checks on a couple students to make sure that we have the right information
# (modify the SSID and check that all fields are valid)
calpads_celdt_df.ix[(calpads_celdt_df['SSID'] == '1096623505') &
                    (calpads_celdt_df['TestNameSpecific'] != 'Comprehension')]\
[['SSID', 'SSIDAdminDate', 'TestNameSpecific', 'ScaleScoreAll', 'TestTakenGrade']]

,SSID,SSIDAdminDate,TestNameSpecific,ScaleScoreAll,TestTakenGrade
2947,1096623505,2015-08-31,Listening,555,6
2948,1096623505,2015-08-31,Overall,515,6
2949,1096623505,2015-08-31,Reading,503,6
2950,1096623505,2015-08-31,Speaking,487,6
2951,1096623505,2015-08-31,Writing,515,6
2953,1096623505,2014-08-21,Listening,541,5
2954,1096623505,2014-08-21,Overall,523,5
2955,1096623505,2014-08-21,Reading,510,5
2956,1096623505,2014-08-21,Speaking,488,5
2957,1096623505,2014-08-21,Writing,556,5


In [259]:
# output the file to csv if we want to do the below manipulations in Excel rather than in this notebook
output_df.to_csv('./data/most_recent_celdt_scores_all_schools.csv', index=False)

## Part 2 - Updating the CELDT DRM Offline Files

1. Load the data, concatenate it (we will make one large file for all schools and upload it to all sites since it only updates the matching eds ids)
2. Merge in the missing values for the student's most recent CELDT from part 1 above.
3. Create fixed-width text file output that can be uploaded to the CELDT DRM tool.

### TODO prior to running the following:
* Download the excel formatted offline CELDT DRM files from the CELDT online portal

In [261]:
# read in all files and concatenate them. reset the index so we can do joins easily later
frames = []

# We choose to deal with all values as strings because the ultimate output is in a string
# if we did not do this step, we would have to deal with formatting at the output rather than the input
converters={
    'edsID1': str,
  'fill0': str,
  'cdsCode': str,
  'charterNum': str,
  'lnameFlag': str,
  'fnameFlag': str,
  'miFlag': str,
  'birthMonth': str,
  'bmonthFlag': str,
  'birthDay': str,
  'bdayFlag': str,
  'birthYear': str,
  'byearFlag': str,
  'gradeSpan': str,
  'grade': str,
  'gradeFlag': str,
  'testPurpose': str,
  'purposeFlag': str,
  'testDate': str,
  'testdateFlag': str,
  'edsID2': str,
  'SSID': str
}

# load all the drm files
for file in glob('./data/drm_excel_files/*.xlsx'):
    dirname, filename = os.path.split(file)
    temp_df = pd.DataFrame(pd.read_excel(file, converters=converters))
    #temp_df['Application Year'] = '20' + filename[0:2] + '-' + filename[2:4]
    frames.append(temp_df)
    
drm_df = pd.concat(frames)
drm_df.reset_index(inplace=True)
del drm_df['index']

# get rid of any full columns of nans (this happens based on the drm file export)
drm_df.dropna(how='all', inplace=True)

# check number of records against number of SSIDS - the two should match in this case
# (they actually don't b/c one student apparently took the test twice this year)
print('Num Records: ' + str(drm_df.shape[0]))
print('Num Unique SSIDs: ' + str(drm_df.drop_duplicates(subset='SSID').shape[0]))

# check inferred dtypes, at this point, all should be dtype('O')
drm_df.columns.to_series().groupby(drm_df.dtypes).groups

Num Records: 341
Num Unique SSIDs: 340


{dtype('O'): ['edition',
  'edsID1',
  'fill0',
  'districtName',
  'cdsCode',
  'charterNum',
  'schoolName',
  'studentLName',
  'lnameFlag',
  'studentFName',
  'fnameFlag',
  'studentMInitial',
  'miFlag',
  'birthMonth',
  'bmonthFlag',
  'birthDay',
  'bdayFlag',
  'birthYear',
  'byearFlag',
  'gender',
  'localID',
  'SSID',
  'ssidFlag',
  'ssidFlagCode',
  'gradeSpan',
  'grade',
  'gradeFlag',
  'testPurpose',
  'purposeFlag',
  'testDate',
  'testdateFlag',
  'prevTestDate',
  'prevDateFlag',
  'prevListenSS',
  'prevLSSFlag',
  'prevSpeakSS',
  'prevSPSSFlag',
  'prevReadSS',
  'prevRSSFlag',
  'prevWritSS',
  'prevWSSFlag',
  'prevOverallSS',
  'prevOSSFlag',
  'prevGrade',
  'prevGradeFlag',
  'accListen',
  'accSpeak',
  'accRead',
  'accWrite',
  'modListen',
  'modSpeak',
  'modRead',
  'modWrite',
  'aaListen',
  'aaSpeak',
  'aaRead',
  'aaWrite',
  'edsID2']}

In [262]:
# check fo any missing SSIDs
drm_df[drm_df['SSID'].isnull()]

,edition,edsID1,fill0,districtName,cdsCode,charterNum,schoolName,studentLName,lnameFlag,studentFName,...,accWrite,modListen,modSpeak,modRead,modWrite,aaListen,aaSpeak,aaRead,aaWrite,edsID2


In [263]:
# check if there are any non 10-digit SSIDs
drm_df[-drm_df['SSID'].str.contains('^[0-9]{10}$')]

,edition,edsID1,fill0,districtName,cdsCode,charterNum,schoolName,studentLName,lnameFlag,studentFName,...,accWrite,modListen,modSpeak,modRead,modWrite,aaListen,aaSpeak,aaRead,aaWrite,edsID2


### Check for valid SSIDs against Illuminate
The next couple of cells will do this. We ideally would pull from the Illuminate DB directly, but can also do it by exporting from Student Search 2.0 the list of all students enrolled in the current academic year.

**Make sure that the `illuminate_academic_year` variable is set correctly if pulling from the DB**

In [296]:
illuminate_academic_year = 2017

query = '''SELECT DISTINCT
  ssc.academic_year,
stud.last_name AS "Last Name",
stud.first_name AS "First Name",
sites.site_name AS "Site Name",
ssc.grade_level_id - 1 AS "Grade",
stud.local_student_id AS "Local Student ID",
stud.state_student_id AS "SSID",
/*stud.english_proficiency,
cep.code_id,*/
cep.code_translation AS "English Proficiency",
/*stud.primary_language,
cl.code_id,*/
cl.code_translation AS "Primary Language",
LOWER(stud.email) AS "Student Email"

FROM
matviews.ss_cube as ssc
  LEFT JOIN public.students stud USING (student_id)
  LEFT JOIN public.sites USING (site_id)
  LEFT JOIN codes.english_proficiency cep ON (cep.code_id = stud.english_proficiency)
  LEFT JOIN codes.language cl ON (cl.code_id = stud.primary_language)
WHERE
  ssc.academic_year = ''' + str(illuminate_academic_year) + ''' AND
  ssc.site_id not in (9999999, 9999998)'''

In [297]:
from sps_automation import sps_automation as sps
sps.load_config()

# create illuminate db connection
db = sps.connect_to_ca_illuminate_db()

# query the data into a Pandas DataFrame
illuminate_df = pd.read_sql(query, db['connection'])

# close db connection
db['connection'].close()

In [264]:
# back up method in case can't connect to Illuminate DB

# read in all files and concatenate them. reset the index so we can do joins easily later
frames = []

for file in glob('./data/illuminate_export/*.xls'):
    dirname, filename = os.path.split(file)
    temp_df = pd.DataFrame(pd.read_excel(file, converters={'State Student ID':str}))
    #temp_df['Application Year'] = '20' + filename[0:2] + '-' + filename[2:4]
    frames.append(temp_df)
    
illuminate_df = pd.concat(frames)
illuminate_df.reset_index(inplace=True)
del illuminate_df['index']

illuminate_df.rename(columns={'State Student ID': 'SSID'}, inplace=True)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [298]:
# get rid of any full columns of nans (this happens based on the drm file export)
illuminate_df.dropna(how='all', inplace=True)

# check number of records against number of SSIDS - the two may not match if there are multiple enrollment records
# per student - say if a student were at multiple sites
print('Num Records: ' + str(illuminate_df.shape[0]))
print('Num Unique SSIDs: ' + str(illuminate_df.drop_duplicates(subset='SSID').shape[0]))

# drop any duplicate SSIDs - it doesn't matter which b/c we are just interested in what the list of SSIDs is
# from Illuminate at this point
illuminate_df.drop_duplicates(subset='SSID', inplace=True)

# number of records should now be equal to number of SSIDs
print('Num Records: ' + str(illuminate_df.shape[0]))
print('Num Unique SSIDs: ' + str(illuminate_df.drop_duplicates(subset='SSID').shape[0]))

# check inferred dtypes
illuminate_df.columns.to_series().groupby(illuminate_df.dtypes).groups

Num Records: 2878
Num Unique SSIDs: 2862
Num Records: 2862
Num Unique SSIDs: 2862


{dtype('int64'): ['academic_year', 'Grade'],
 dtype('O'): ['Last Name',
  'First Name',
  'Site Name',
  'Local Student ID',
  'SSID',
  'English Proficiency',
  'Primary Language',
  'Student Email']}

In [266]:
# show records for students with state student ids from CELDT that don't match SSIDs from Illuminate
matched_df = pd.merge(drm_df, illuminate_df, on='SSID', how='left')
matched_df[matched_df['Student ID'].isnull()][['SSID', 'Student ID', 'districtName', 'studentLName', 'studentFName', 'studentMInitial', 'grade']]

,SSID,Student ID,districtName,studentLName,studentFName,studentMInitial,grade
274,6116047332,NaN,SUMMIT PUBLIC SCHOOL: SHASTA,ZENG,PANDORA,,09


In [267]:
# update any incorrect SSIDs from the above cell
drm_df.ix[drm_df['SSID'] == '6116047332', 'SSID'] = '6116047334'

In [268]:
# check for any duplicate SSIDs
# if there are, manually check these for which score is higher - this might indicate
# that a student tested twice
drm_df[drm_df.duplicated(subset='SSID')]

,edition,edsID1,fill0,districtName,cdsCode,charterNum,schoolName,studentLName,lnameFlag,studentFName,...,accWrite,modListen,modSpeak,modRead,modWrite,aaListen,aaSpeak,aaRead,aaWrite,edsID2
183,2016-2017,021610709002802,NaN,SUMMIT PREPARATORY CHARTER HI,41690620112722,0835,SUMMIT PREPARATORY CHARTER HI,HERNANDEZ,NaN,OSCAR,...,,,,,,,,,,021610709002802


In [269]:
# after SSID cleaning, merge the prev scores that we got in output_df  from CALPADS into the drm_df and 
# fix the columns
final_drm_df = pd.merge(drm_df, output_df, on='SSID', how='left')

In [270]:
# update the column values appropriately, then delete
final_drm_df['prevTestDate'] = final_drm_df['SSIDAdminDate'].dt.strftime('%m%Y')
final_drm_df['prevListenSS'] = final_drm_df['Listening']
final_drm_df['prevSpeakSS'] = final_drm_df['Speaking']
final_drm_df['prevReadSS'] = final_drm_df['Reading']
final_drm_df['prevWritSS'] = final_drm_df['Writing']
final_drm_df['prevOverallSS'] = final_drm_df['Overall']
final_drm_df['prevGrade'] = final_drm_df['TestTakenGrade']



In [271]:
# remove the excess columns to get back to the original drm file state
for col in ['SSIDAdminDate', 'Listening','Speaking','Reading','Writing','Overall', 'TestTakenGrade']:
    del final_drm_df[col]

In [247]:
final_drm_df.to_csv('test_date.csv', index=False)

## Output: Fixed-Width Text File for CELDT DRM Upload

Get the widths from the [2016-17 DRM Data File Layout]('https://www.celdt.org/documents/2016-17/CELDT_DRM_Data_File_Layout_2016-17.pdf')

In [272]:
# transcribe widths of each field
fw_col_widths = [9,15,1,30,14,4,30,11,1,9,1,1,1,2,1,2,1,4,1,1,10,10,1,1,1,2,1,1,1,8,1,6,1,3,1,3,1,3,1,3,1,3,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,15]

# Check that the length sums to 233
sum(fw_col_widths)

233

In [162]:
len(final_drm_df.values.tolist())

341

In [286]:
final_drm_df.fillna('').values.tolist()[1].index('3')

43

In [287]:
fw_output_list = list()
for row in final_drm_df.fillna('').values.tolist():
    # NaTs can't be filled with a string like '', so they are not caught by fillna above
    # We actually have to catch the text and replace it
    if row[31] == 'NaT':
        row[31] = ''

    # we need a two-digit grade value (precede 0-9 with a zero e.g. 09)
    # TODO change to regex
    if row[43] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']:
        row[43] = '0' + row[43]
    
    # the following takes each row of the DF and inserts it as a string with the proper output width
    fw_output_list.append("".join("%-*s" % i for i in zip(fw_col_widths, row)))

In [291]:
# check that we still have the correct number of students in the final list
len(fw_output_list)

341

In [292]:
# output the final fixed-width text document that we can upload to each one of the CELDT portal DRM modules
fw_output_filename = 'celdt_drm_fw_upload.txt'
with open('./' + fw_output_filename, mode='wt', encoding='utf-8') as out_file:
    out_file.write('\n'.join(fw_output_list))